In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
import re
import urllib
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 

In [2]:
df=pd.read_excel(r'C:\Users\pinup\Downloads\cik_list.xlsx')

In [3]:
df['SECFNAME']='https://www.sec.gov/Archives/'+df['SECFNAME'].astype(str)

In [4]:
stop_words=[]
f=open(r"C:\Users\pinup\Downloads\StopWords_GenericLong.txt",'r')
for i in f:
    i=i.strip()
    stop_words.append(i)

In [5]:
df_p=pd.read_excel(r"C:\Users\pinup\Downloads\LoughranMcDonald_SentimentWordLists_2018.xlsx",sheet_name='Positive',header=None)

In [6]:
df_N=pd.read_excel(r"C:\Users\pinup\Downloads\LoughranMcDonald_SentimentWordLists_2018.xlsx",sheet_name='Negative',header=None)

In [7]:
df_p.rename( columns={0:'postives'}, inplace=True )
df_N.rename(columns={0:'negatives'},inplace=True)

In [8]:
positive_words=df_p['postives'].tolist()
negative_words=df_N['negatives'].tolist()
positive_words=[w.lower() for w in positive_words]
negative_words=[w.lower() for w in negative_words]

In [148]:
def filtering(link):
    matter=[]
    mg=[]
    qc=[]
    rf=[]
    i=None
    j=None
    p=None
    file = urllib.request.urlopen(link)
    for line in file:
        decoded_line = line.decode("utf-8")
        clean=re.compile('<.*?>')
        clean_line=re.sub(clean,'',decoded_line)
        matter.append(clean_line)
    for index,cont in enumerate(matter):
        r=cont.find("MANAGEMENT'S DISCUSSION")
        t=cont.find("QUANTITATIVE AND QUALITATIVE")
        e=cont.find("RISK FACTORS")
        if r!=-1:
            i=index
        if t!=-1:
            j=index
        if e!=-1:
            p=index
    if i!=None:
        for k in range(i,len(matter)-1):
            g=matter[k+1].find("ITEM")
            mg.append(matter[k])
            if g!=-1:
                break
    if j!=None:
        for k in range(j,len(matter)-1):
            g=matter[k+1].find("ITEM")
            qc.append(matter[k])
            if g!=-1:
                break 
    if p!=None:
        for k in range(p,len(matter)-1):
            g=matter[k+1].find("ITEM")
            rf.append(matter[k])
            if g!=-1:
                break
    return(mg,qc,rf)

In [149]:
def processing(m,q,r):
    pos_m=0
    pos_q=0
    pos_r=0
    neg_m=0
    neg_q=0
    neg_r=0
    required_m=[]
    required_q=[]
    required_r=[]
    pos_prop_m=[]
    pos_prop_q=[]
    pos_prop_r=[]
    neg_prop_m=[]
    neg_prop_q=[]
    neg_prop_r=[]
    str_m=''.join(m)
    str_q=''.join(q)
    str_r=''.join(r)
    tokens_m=word_tokenize(str_m)
    tokens_q=word_tokenize(str_q)
    tokens_r=word_tokenize(str_r)
    tokens_m= [word for word in tokens_m if word.isalnum()]
    tokens_q= [word for word in tokens_q if word.isalnum()]
    tokens_r= [word for word in tokens_r if word.isalnum()]
    tokens_m = [w.lower() for w in tokens_m]
    tokens_q = [w.lower() for w in tokens_q]
    tokens_r = [w.lower() for w in tokens_r]
    for i in tokens_m:
        if i in stop_words:
            pass
        else:
            required_m.append(i)
            
    for i in tokens_q:
        if i in stop_words:
            pass
        else:
            required_q.append(i)
    for i in tokens_r:
        if i in stop_words:
            pass
        else:
            required_r.append(i)
    m_score=[]
    q_score=[]
    r_score=[]
    for i in required_m:
        if i in positive_words:
            pos_m=pos_m+1
        if i in negative_words:
            neg_m=neg_m-1
    m_score.append((pos_m,neg_m*(-1),len(required_m)))                 
    for i in required_q:
        if i in positive_words:
            pos_q=pos_q+1
        if i in negative_words:
            neg_q=neg_q-1
    q_score.append((pos_q,neg_q*(-1),len(required_q)))      
    for i in required_r:
        if i in positive_words:
            pos_r=pos_r+1
        if i in negative_words:
            neg_r=neg_r-1
    r_score.append((pos_r,neg_r*(-1),len(required_r)))
    m_polarity=[]
    q_polarity=[]
    r_polarity=[]
    for i in range(len(m_score)):
        try:
            for i in m_score:
                m_polarity.append((i[0]-i[1])/((i[0]+i[1])+0.000001))
        except ZeroDivisionError:
            m_polarity.append(0)
    for i in range(len(q_score)):
        try:
            for i in q_score:
                q_polarity.append((i[0]-i[1])/((i[0]+i[1])+0.000001))
        except ZeroDivisionError:
             q_polarity.append(0)
    for i in range(len(r_score)):
        try:
            for i in r_score:
                r_polarity.append((i[0]-i[1])/((i[0]+i[1])+0.000001))
        except ZeroDivisionError:
             r_polarity.append(0)
    m_subjectivity=[]
    q_subjectivity=[]
    r_subjectivity=[]
    for i in range(len(m_score)):
        try:
            for i in m_score:
                m_subjectivity.append((i[0] + i[1])/(i[2]+0.000001))
        except ZeroDivisionError:
            m_subjectivity.append(0)
            
    for i in range(len(q_score)):
        try:
            for i in q_score:
                q_subjectivity.append((i[0] + i[1])/(i[2]+0.000001))
        except ZeroDivisionError:
            q_subjectivity.append(0)
            
    for i in range(len(r_score)):
        try:
            for i in r_score:
                r_subjectivity.append((i[0] + i[1])/(i[2]+0.000001))
        except ZeroDivisionError:
            r_subjectivity.append(0)
    for i in range(len(m_score)):
        try:
            for i in m_score:
                pos_prop_m.append(i[0]/len(tokens_m))
        except:
            pos_prop_m.append(0)
    for i in range(len(q_score)):
        try:
            for i in q_score:
                pos_prop_q.append(i[0]/len(tokens_q))
        except:
            pos_prop_q.append(0)
    for i in range(len(r_score)):
        try:
            for i in r_score:
                pos_prop_r.append(i[0]/len(tokens_r))
        except:
            pos_prop_r.append(0)
            
    for i in range(len(m_score)):
        try:
            for i in m_score:
                neg_prop_m.append(i[1]/len(tokens_m))
        except:
            neg_prop_m.append(0)
    for i in range(len(q_score)):
        try:
            for i in q_score:
                neg_prop_q.append(i[1]/len(tokens_q))
        except:
            neg_prop_q.append(0)
    for i in range(len(r_score)):
        try:
            for i in r_score:
                neg_prop_r.append(i[1]/len(tokens_r))
        except:
            neg_prop_r.append(0)
    return(m_score,q_score,r_score,m_polarity,q_polarity,r_polarity,pos_prop_m,pos_prop_q,pos_prop_r,neg_prop_m,neg_prop_q,neg_prop_r)

In [150]:
def avg_sentence(m,q,r):
    str_m=''.join(m)
    str_q=''.join(q)
    str_r=''.join(r)
    tokens_m=word_tokenize(str_m)
    tokens_q=word_tokenize(str_q)
    tokens_r=word_tokenize(str_r)
    tokens_m= [word for word in tokens_m if word.isalpha()]
    tokens_q= [word for word in tokens_q if word.isalpha()]
    tokens_r= [word for word in tokens_r if word.isalpha()]
    tokens_sm=sent_tokenize(str_m)
    tokens_sq=sent_tokenize(str_q)
    tokens_sr=sent_tokenize(str_r)
    try:
        avg_sen_m=len(tokens_m)/len(tokens_sm)
    except:
        avg_sen_m=0
    try:
        avg_sen_q=len(tokens_q)/len(tokens_sq)
    except:
        avg_sen_q=0
    try:
        avg_sen_r=len(tokens_r)/len(tokens_sr)
    except:
        avg_sen_r=0
    return(avg_sen_m,avg_sen_q,avg_sen_r)

In [151]:
def per_complex_words(m,q,r):
    count=0
    m_count=0
    q_count=0
    r_count=0
    vowels=['a','e','i','o','u']
    str_m=''.join(m)
    str_q=''.join(q)
    str_r=''.join(r)
    tokens_m=word_tokenize(str_m)
    tokens_q=word_tokenize(str_q)
    tokens_r=word_tokenize(str_r)
    tokens_m= [word for word in tokens_m if word.isalnum()]
    tokens_q= [word for word in tokens_q if word.isalnum()]
    tokens_r= [word for word in tokens_r if word.isalnum()]
    tokens_m = [w.lower() for w in tokens_m]
    tokens_q = [w.lower() for w in tokens_q]
    tokens_r = [w.lower() for w in tokens_r]
    for i in tokens_m:
        if i.endswith('ed') or i.endswith('es'):
            pass
        else:
            for j in range(len(i)):
                if i[j] in vowels:
                    count+=1
                    if count>2:
                        m_count+=1
                        count=0
    count=0
    for i in tokens_q:
        if i.endswith('ed') or i.endswith('es'):
            pass
        else:
            for j in range(len(i)):
                if i[j] in vowels:
                    count+=1
                    if count>2:
                        q_count+=1
                        count=0
    count=0
    for i in tokens_r:
        if i.endswith('ed') or i.endswith('es'):
            pass
        else:
            for j in range(len(i)):
                if i[j] in vowels:
                    count+=1
                    if count>2:
                        r_count+=1
                        count=0
    try:
        per_m=m_count/len(tokens_m)
    except:
        per_m=0
    try:
        per_q=q_count/len(tokens_q)
    except:
        per_q=0
    try:
        per_r=r_count/len(tokens_r)
    except:
        per_r=0
    return(len(tokens_m),len(tokens_q),len(tokens_r),m_count,q_count,r_count,per_m,per_q,per_r)

In [152]:
def fog_index(avg_sen,per_com):
    fog_m=0.4*(avg_sen[0]+per_com[0])
    fog_q=0.4*(avg_sen[1]+per_com[1])
    fog_r=0.4*(avg_sen[2]+per_com[2])    
    return(fog_m,fog_q,fog_r)

In [153]:
df_un=pd.read_excel(r'C:\Users\pinup\Downloads\uncertainty_dictionary.xlsx')
df_con=pd.read_excel(r'C:\Users\pinup\Downloads\constraining_dictionary.xlsx')

In [154]:
un_words=df_un['Word'].tolist()
con_words=df_con['Word'].tolist()
un_words=[w.lower() for w in un_words]
con_words=[w.lower() for w in con_words]

In [163]:
def un_certainty(m,q,r):
    count_m=0
    count_q=0
    count_r=0
    un_prop_m=0
    un_prop_q=0
    un_prop_r=0
    str_m=''.join(m)
    str_q=''.join(q)
    str_r=''.join(r)
    tokens_m=word_tokenize(str_m)
    tokens_q=word_tokenize(str_q)
    tokens_r=word_tokenize(str_r)
    tokens_m= [word for word in tokens_m if word.isalnum()]
    tokens_q= [word for word in tokens_q if word.isalnum()]
    tokens_r= [word for word in tokens_r if word.isalnum()]
    tokens_m = [w.lower() for w in tokens_m]
    tokens_q = [w.lower() for w in tokens_q]
    tokens_r = [w.lower() for w in tokens_r]
    for i in tokens_m:
        if i in un_words:
            count_m+=1
    for i in tokens_q:
        if i in un_words:
            count_q+=1
    for i in tokens_r:
        if i in un_words:
            count_r+=1         
    try:
        un_prop_m=count_m/(len(tokens_m))
    except:
        un_prop_m=0
    try:
        un_prop_q=count_q/(len(tokens_q))
    except:
        un_prop_q=0
    try:
        un_prop_r=count_r/(len(tokens_r))
    except:
        un_prop_r=0
    return(count_m,count_q,count_r,un_prop_m,un_prop_q,un_prop_r)

In [156]:
def constrain(m,q,r):
    count_m=0
    count_q=0
    count_r=0
    str_m=''.join(m)
    str_q=''.join(q)
    str_r=''.join(r)
    tokens_m=word_tokenize(str_m)
    tokens_q=word_tokenize(str_q)
    tokens_r=word_tokenize(str_r)
    tokens_m= [word for word in tokens_m if word.isalnum()]
    tokens_q= [word for word in tokens_q if word.isalnum()]
    tokens_r= [word for word in tokens_r if word.isalnum()]
    tokens_m = [w.lower() for w in tokens_m]
    tokens_q = [w.lower() for w in tokens_q]
    tokens_r = [w.lower() for w in tokens_r]
    for i in tokens_m:
        if i in con_words:
            count_m+=1
    for i in tokens_q:
        if i in con_words:
            count_q+=1
    for i in tokens_r:
        if i in con_words:
            count_r+=1
    try:
        con_prop_m=count_m/(len(tokens_m))
    except:
        con_prop_m=0
    try:
        con_prop_q=count_q/(len(tokens_q))
    except:
        con_prop_q=0
    try:
        con_prop_r=count_r/(len(tokens_r))
    except:
        con_prop_r=0
    return(count_m,count_q,count_r,con_prop_m,con_prop_q,con_prop_r)

In [157]:
def w_uncer_cons(link):
    count_un=0
    count_con=0
    file = urllib.request.urlopen(link)
    for line in file:
        decoded_line = line.decode("utf-8")
        clean=re.compile('<.*?>')
        clean_line=re.sub(clean,'',decoded_line)
        tokens=word_tokenize(clean_line)
        tokens= [word for word in tokens if word.isalnum()]
        tokens = [w.lower() for w in tokens]
        for i in tokens:
            if i in un_words:
                count_un+=1
            if i in con_words:
                count_con+=1
    return(count_un,count_con)

In [158]:
result=[]

In [159]:
len(result)

0

In [164]:
m_positive=[]
q_positive=[]
r_positive=[]
m_negative=[]
q_negative=[]
r_negative=[]
m_polarity=[]
q_polarity=[]
r_polarity=[]
m_avg_sent=[]
q_avg_sent=[]
r_avg_sent=[]
m_per_comp=[]
q_per_comp=[]
r_per_comp=[]
m_fog_ind=[]
q_fog_ind=[]
r_fog_ind=[]
m_com_count=[]
q_com_count=[]
r_com_count=[]
m_word_count=[]
q_word_count=[]
r_word_count=[]
m_uncertain=[]
q_uncertain=[]
r_uncertain=[]
m_constrain=[]
q_constrain=[]
r_constrain=[]
w_uncertain=[]
w_constrain=[]
m_pos_proportion=[]
q_pos_proportion=[]
r_pos_proportion=[]
m_neg_proportion=[]
q_neg_proportion=[]
r_neg_proportion=[]
m_un_proportion=[]
q_un_proportion=[]
r_un_proportion=[]
m_con_proportion=[]
q_con_proportion=[]
r_con_proportion=[]

In [165]:
for i in df['SECFNAME']:
    z=filtering(i)
    pro=processing(z[0],z[1],z[2])
    a=avg_sentence(z[0],z[1],z[2])
    p=per_complex_words(z[0],z[1],z[2])
    f=fog_index(a,p)
    u=un_certainty(z[0],z[1],z[2])
    c=constrain(z[0],z[1],z[2])
    unc=w_uncer_cons(i)
    pro_mp=pro[0][0][0]
    pro_qp=pro[1][0][0]
    pro_rp=pro[2][0][0]
    pro_mn=pro[0][0][1]
    pro_qn=pro[1][0][1]
    pro_rn=pro[2][0][1]
    pro_mpol=pro[3][0]
    pro_qpol=pro[4][0]
    pro_rpol=pro[5][0]
    avg_sen_m=a[0]
    avg_sen_q=a[1]
    avg_sen_r=a[2]
    m_positive.append(pro_mp)
    q_positive.append(pro_qp)
    r_positive.append(pro_rp)
    m_negative.append(pro_mn)
    q_negative.append(pro_qn)
    r_negative.append(pro_rn)
    m_polarity.append(pro_mpol)
    q_polarity.append(pro_qpol)
    r_polarity.append(pro_rpol)
    m_avg_sent.append(avg_sen_m)
    q_avg_sent.append(avg_sen_q)
    r_avg_sent.append(avg_sen_r)
    m_word_count.append(p[0])
    q_word_count.append(p[1])
    r_word_count.append(p[2])
    m_com_count.append(p[3])
    q_com_count.append(p[4])
    r_com_count.append(p[5])
    m_per_comp.append(p[6])
    q_per_comp.append(p[7])
    r_per_comp.append(p[8])
    m_fog_ind.append(f[0])
    q_fog_ind.append(f[1])
    r_fog_ind.append(f[2])
    m_uncertain.append(u[0])
    q_uncertain.append(u[1])
    r_uncertain.append(u[2])
    m_constrain.append(c[0])
    q_constrain.append(c[1])
    r_constrain.append(c[2])
    w_uncertain.append(unc[0])
    w_constrain.append(unc[1])
    m_pos_proportion.append(pro[6][0])
    q_pos_proportion.append(pro[7][0])
    r_pos_proportion.append(pro[8][0])
    m_neg_proportion.append(pro[9][0])
    q_neg_proportion.append(pro[10][0])
    r_neg_proportion.append(pro[11][0])
    m_un_proportion.append(u[3])
    q_un_proportion.append(u[4])
    r_un_proportion.append(u[5])
    m_con_proportion.append(c[3])
    q_con_proportion.append(c[3])
    r_con_proportion.append(c[3])

In [168]:
di=df['SECFNAME'].tolist()

In [176]:
output={'cik_list': di,'mda_positive_score': m_positive,'mda_negative_score': m_negative,'mda_polarity_score': m_polarity,'mda_average_sentence_length': m_avg_sent,'mda_percentage_of_complex_words': m_per_comp,'mda_fog_index': m_fog_ind,'mda_complex_word_count': m_com_count,'mda_word_count': m_word_count,'mda_uncertainty_score': m_uncertain,'mda_constraining_score': m_constrain,'mda_positive_word_proportion': m_pos_proportion,'mda_negative_word_proportion': m_neg_proportion,'mda_uncertainty_word_proportion': m_un_proportion,'mda_constraining_word_proportion': m_con_proportion,'qqdmr_positive_score': q_positive,'qqdmr_negative_score': q_negative,'qqdmr_polarity_score': q_polarity,'qqdmr_average_sentence_length':q_avg_sent,'qqdmr_percentage_of_complex_words':q_per_comp,'qqdmr_fog_index':q_fog_ind,'qqdmr_complex_word_count':q_com_count,'qqdmr_word_count':q_word_count,'qqdmr_uncertainty_score':q_uncertain,'qqdmr_constraining_score':q_constrain,'qqdmr_positive_word_proportion':q_pos_proportion,'qqdmr_negative_word_proportion':q_neg_proportion,'qqdmr_uncertainty_word_proportion':q_un_proportion,'qqdmr_constraining_word_proportion':q_con_proportion,'rf_positive_score':r_positive,'rf_negative_score':r_negative,'rf_polarity_score':r_polarity,'rf_average_sentence_length':r_avg_sent,'rf_percentage_of_complex_words':r_per_comp,'rf_fog_index':r_fog_ind,'rf_complex_word_count':r_com_count,'rf_word_count':r_word_count,'rf_uncertainty_score':r_uncertain,'rf_constraining_score':r_constrain,'rf_positive_word_proportion':r_pos_proportion,'rf_negative_word_proportion':r_neg_proportion,'rf_uncertainty_word_proportion':r_un_proportion,'rf_constraining_word_proportion':r_con_proportion,'constraining_words_whole_report':w_constrain}

In [177]:
output_pd=pd.DataFrame(output)

In [179]:
output_pd.to_csv('output_dict.csv')